In [0]:
!wget -O aliza.zip https://doc-0k-90-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mceo9tkji8qaue191fo0i6snbodvp177/1580112000000/09750456226342476441/*/15otGqHvpdUgwGVs_hJKqnQ3U3LrVcaE0?e=download

--2020-01-27 09:23:27--  https://doc-0k-90-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mceo9tkji8qaue191fo0i6snbodvp177/1580112000000/09750456226342476441/*/15otGqHvpdUgwGVs_hJKqnQ3U3LrVcaE0?e=download
Resolving doc-0k-90-docs.googleusercontent.com (doc-0k-90-docs.googleusercontent.com)... 74.125.31.132, 2607:f8b0:400c:c02::84
Connecting to doc-0k-90-docs.googleusercontent.com (doc-0k-90-docs.googleusercontent.com)|74.125.31.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘aliza.zip’

aliza.zip               [        <=>         ] 665.39M   108MB/s    in 6.3s    

2020-01-27 09:23:34 (106 MB/s) - ‘aliza.zip’ saved [697712823]



In [0]:
!unzip aliza.zip

Archive:  aliza.zip
   creating: aliza/
   creating: aliza/fifty/
   creating: aliza/ten/
   creating: aliza/twenty/
  inflating: aliza/twenty/tw_1370.jpg  
  inflating: aliza/twenty/tw_1711.jpg  
  inflating: aliza/twenty/tw_1244.jpg  
  inflating: aliza/twenty/tw_1035.jpg  
  inflating: aliza/twenty/tw_2023.jpg  
  inflating: aliza/twenty/tw_2084.jpg  
  inflating: aliza/twenty/tw_1996.jpg  
  inflating: aliza/twenty/tw_826.jpg  
  inflating: aliza/twenty/tw_1781.jpg  
  inflating: aliza/twenty/tw_1843.jpg  
  inflating: aliza/twenty/tw_1408.jpg  
  inflating: aliza/twenty/tw_2547.jpg  
  inflating: aliza/twenty/tw_1228.jpg  
  inflating: aliza/twenty/tw_2453.jpg  
  inflating: aliza/twenty/tw_1482.jpg  
  inflating: aliza/twenty/tw_1214.jpg  
  inflating: aliza/twenty/tw_1805.jpg  
  inflating: aliza/twenty/tw_867.jpg  
  inflating: aliza/twenty/tw_771.jpg  
  inflating: aliza/twenty/tw_875.jpg  
  inflating: aliza/twenty/tw_1808.jpg  
  inflating: aliza/twenty/tw_921.jpg  
  inflat

In [0]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

DATADIR="aliza"
CATEGORIES=["ten","twenty","fifty"]
img_size=256


In [0]:
from google.colab import files

with open('model1.hdf5', 'w') as f:
  f.write('model.hdf5')

files.download('model1.hdf5')

In [6]:
#For preprocessing of the data
training_data=[]
def create_training_data():
  for i in tqdm(CATEGORIES):
    path=os.path.join(DATADIR,i)
    class_num=CATEGORIES.index(i)
    print(class_num)
    for img in (os.listdir(path)):
      img_array=cv.resize(cv.imread(os.path.join(path,img),cv.IMREAD_COLOR),(img_size,img_size))
      training_data.append([img_array,class_num])
  np.save('train_data.npy',training_data)
  random.shuffle(training_data)       
create_training_data()


  0%|          | 0/3 [00:00<?, ?it/s]

0


 33%|███▎      | 1/3 [00:13<00:26, 13.42s/it]

1


 67%|██████▋   | 2/3 [00:24<00:12, 12.86s/it]

2


100%|██████████| 3/3 [00:38<00:00, 13.17s/it]


In [7]:
X =[]
y =[]
for features,label in training_data:
    X.append(features)
    y.append(label)
X=np.array(X)
y=np.array(y)
print(y)

[0 1 0 ... 1 2 1]


In [8]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from keras.utils import to_categorical  
from tensorflow.python.keras.optimizers import Adam, Adagrad

#NAME="currencyy"
#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
X = X.reshape([-1,img_size,img_size, 3])

X=X/255.0    #normalizing features
y= to_categorical(y)
print(y)

model = Sequential()

model.add(Conv2D(256,kernel_size=(3, 3),strides=(1,1),input_shape=(img_size,img_size,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256,kernel_size=(3, 3),strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(3))
model.add(Activation('softmax'))
opt=Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])#binary for using only two classes

model.fit(X, y, batch_size=32, epochs=4, validation_split=0.2)
model.save('model1.hdf5')


Using TensorFlow backend.


[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 4934 samples, validate on 1234 samples
Epoch 1/4
4934/4934 [==============================] - 49s 10ms/sample - loss: 0.7265 - acc: 0.6782 - val_loss: 0.4644 - val_acc: 0.7998
Epoch 2/4
4934/4934 [==============================] - 41s 8ms/sample - loss: 0.3034 - acc: 0.8887 - val_loss: 0.2257 - val_acc: 0.9287
Epoch 3/4
4934/4934 [==============================] - 42s 8ms/sample - loss: 0.1665 - acc: 0.9467 - val_loss: 0.1814 - val_acc: 0.9408
Epoch 4/4
4934/4934 [==============================] - 42s 8ms/sample - loss: 0.1026 - acc: 0.9714 - val_loss: 0.1279 - val_acc: 0.9595


In [9]:
model.summary() #for summary or detail of the model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 256)     7168      
_________________________________________________________________
activation (Activation)      (None, 254, 254, 256)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 256)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 256)     590080    
_________________________________________________________________
activation_1 (Activation)    (None, 125, 125, 256)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 984064)            0

In [0]:
def prepare(filepath):
    img_array = cv.imread(filepath, cv.IMREAD_COLOR)  # read in the image
    new_array = cv.resize(img_array, (img_size,img_size))# resize image to match model's expected sizing
    new_array=new_array/1.0
    return new_array.reshape(-1,img_size,img_size, 3)  # return the image with shaping that TF wants.


In [18]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

model = tf.keras.models.load_model("model1.hdf5")

prediction = model.predict(prepare('3.jpg'))
print(prediction)
if prediction[0][0]== [1]:
  print("ten")
elif prediction[0][1]==[1]:
  print("twenty")
elif prediction[0][2]==[1]:
  print("fifty")


[[0. 1. 0.]]
twenty
